In [ ]:
import os
!git clone https://github.com/Maria-Elisa-M/BW_prediction_keypoint.git
os.chdir('BW_prediction_keypoint')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install imagecodecs

In [3]:

import os
import pandas as pd
import numpy as np
import skimage.io
from skimage.measure import label, regionprops
from skimage.io import imsave
import tifffile
import matplotlib.pyplot as plt
from skimage.transform import rotate
from scipy import ndimage as nd
import math
import imagecodecs

# FUNCTIONS

In [4]:

def extract_cow_day(filename):
    """Extracts cow ID and day from the filename, assuming format 'cowID_dayXXXXXX.png'"""
    parts = filename.split("_")
    if len(parts) >= 2:
        cow, day = parts[0], parts[1][:8]  # Extract cow ID and first 8 chars of day
    else:
        cow, day = np.nan, np.nan  # Assign NaN if format is incorrect
    return cow, day

def calculate_vertical_center_points(mask, num_points=13):
    """Calculate vertical center points along evenly spaced columns in the mask."""
    labeled_mask = label(mask)
    properties = regionprops(labeled_mask)

    if not properties:
        return []

    minr, minc, maxr, maxc = properties[0].bbox
    spaced_columns = np.linspace(minc, maxc - 1, num=num_points, dtype=int)

    center_points = []
    for c in spaced_columns:
        col = mask[minr:maxr, c]
        if np.any(col):
            top_edge = minr + np.argmax(col)
            bottom_edge = maxr - np.argmax(col[::-1])
            center_y = (top_edge + bottom_edge) // 2
            width = bottom_edge - top_edge
            center_points.append((center_y, c, top_edge, bottom_edge, width))

    return center_points

def calculate_shape_features(mask, depth, floor_distance=2225):
    """Compute shape features based on the mask and depth map."""
    if mask.size == 0:
        return [0] * 7

    pred_mask = mask.astype(bool)

    # Handle invalid depth values
    invalid_cell_mask = depth == 0
    indices = nd.distance_transform_edt(invalid_cell_mask, return_distances=False, return_indices=True)
    depth = depth[tuple(indices)]
    depth[~pred_mask] = 0
    depth[depth >= floor_distance] = 0
    depth_to_floor = floor_distance - depth
    depth_to_floor[depth == 0] = 0

    # Camera parameters
    focal_length = 0.6
    sensor_size = 0.0014
    f = focal_length / sensor_size
    each_pixel_area = (depth / f) ** 2

    # Compute features
    area = np.sum(each_pixel_area) * 0.01  # cm²
    volume = np.sum(each_pixel_area * depth_to_floor) * 1e-6  # L

    labeled_mask = label(pred_mask)
    properties = regionprops(labeled_mask)

    if not properties:
        return [area, volume, 0, 0, 0, 0, 0]

    prop = properties[0]
    circularity = (4 * prop.area * math.pi) / (prop.perimeter ** 2) if prop.perimeter > 0 else 0
    extent = prop.extent
    eccentricity = prop.eccentricity
    perimeter = prop.perimeter

    # Compute major axis length using image rotation
    imgrot = rotate(pred_mask, -prop.orientation * 180 / math.pi, resize=True)
    hor = np.max(np.sum(imgrot, axis=1))
    vert = np.max(np.sum(imgrot, axis=0))
    major_axis_length = max(hor, vert)

    return [area, volume, circularity, extent, eccentricity, perimeter, major_axis_length]

def calculate_distances(depth_map, points):
    """Extract depth values at specified key points."""
    return [depth_map[int(y), int(x)] if (0 <= int(y) < depth_map.shape[0] and 0 <= int(x) < depth_map.shape[1]) else np.nan for y, x in points]

def process_images(image_directory, mask_directory, output_directory):
    """Process depth and mask images, extract features, and save results."""
    os.makedirs(output_directory, exist_ok=True)

    mask_files = [f for f in os.listdir(mask_directory) if f.endswith('.png')]
    depth_files = {f for f in os.listdir(image_directory) if f.endswith('.tif')}

    results = []

    for mask_file in mask_files:
        base_name = os.path.splitext(mask_file)[0]
        depth_file = f"{base_name}.tif"

        if depth_file in depth_files:
            mask_path = os.path.join(mask_directory, mask_file)
            depth_map_path = os.path.join(image_directory, depth_file)

            mask = skimage.io.imread(mask_path)
            depth_map = tifffile.imread(depth_map_path)

            # Extract 'cow' and 'day' from filename
            cow, day = extract_cow_day(mask_file)

            # Compute distances & shape features
            center_points = calculate_vertical_center_points(mask)
            if len(center_points) > 2:
                center_points = center_points[1:-1]

            center_y_points = [(cy, c) for cy, c, _, _, _ in center_points]
            widths = [w for _, _, _, _, w in center_points]

            distances = calculate_distances(depth_map, center_y_points)
            shape_features = calculate_shape_features(mask, depth_map)

            # Save results
            results.append([mask_file, cow, day] + distances + widths + shape_features)

            # Save visualization
            plt.imshow(mask, cmap='gray')
            for cy, c in center_y_points:
                plt.plot(c, cy, 'yo')
            for cy, c, top, bottom, _ in center_points:
                plt.plot([c, c], [top, bottom], 'r-')

            plt.savefig(os.path.join(output_directory, f"{base_name}_graph.png"))
            plt.close()

    # Save to CSV
    df = pd.DataFrame(results, columns=['Image Name', 'cow', 'day'] +
                      [f'Distance_{i+1}' for i in range(11)] +
                      [f'Width_{i+1}' for i in range(11)] +
                      ['Area', 'Volume', 'Circularity', 'Extent', 'Eccentricity', 'Perimeter', 'Major_Axis_Length'])

    df.to_csv(os.path.join(output_directory, 'distances_shape_features_results.csv'), index=False)

def compute_averages(df):
    """Group data by cow and day, computing averages for numeric columns."""
    numeric_columns = df.columns[3:]

    # Convert relevant columns to numeric
    df[numeric_columns] = df[numeric_columns].apply(pd.to_numeric, errors='coerce')

    # Ensure 'cow' and 'day' are strings before grouping
    df[['cow', 'day']] = df[['cow', 'day']].astype(str)

    return df.groupby(['cow', 'day'], as_index=False)[numeric_columns].mean()

def merge_bw_data(df_new, bw_file_path):
    """Merge body weight (BW) data based on cow and day."""
    df_bw = pd.read_csv(bw_file_path, dtype={'day': str, 'cow': str})
    df_new[['day', 'cow']] = df_new[['day', 'cow']].astype(str)
    return df_new.merge(df_bw, on=['cow', 'day'], how='left')


# MAIN EXECUTION

In [ ]:
# Define directories
main_dir = 'depth'
image_directory = os.path.join(main_dir,'Images')

save_dir = os.path.join(main_dir, 'output')
mask_directory = os.path.join(save_dir, 'masks')
output_directory = os.path.join(save_dir,'graphs')

# create the ouput directories
os.makedirs(save_dir, exist_ok=True)
os.makedirs(mask_directory, exist_ok=True)
os.makedirs(output_directory, exist_ok=True)
os.makedirs(save_dir, exist_ok=True)


In [6]:
# Process images
process_images(image_directory, mask_directory, output_directory)



In [7]:
# Load and process data
output_csv_path = os.path.join(output_directory, 'distances_shape_features_results.csv')
df = pd.read_csv(output_csv_path)

# Compute averages
df_new = compute_averages(df)

# Merge BW data
bw_file_path = os.path.join(main_dir,'DF','BW.csv')
df_new = merge_bw_data(df_new, bw_file_path)

# Save final data
output_csv = os.path.join(save_dir, "results_bw.csv")
df_new.to_csv(output_csv, index=False)

print(f"Processing complete. Results saved to {output_csv}.")

Processing complete. Results saved to /content/drive/MyDrive/guilherme/depth/output/results_bw.csv.
